# Processing and deduplicating Features

In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict, Counter

In [29]:
train_df = pd.read_json("../input/train.json")
test_df = pd.read_json("../input/test.json")
train_test = pd.concat([train_df, test_df], 0)

I'm going do assume you've taken a look at the data and noticed that some features are duplicates and that some are also rare (occurs very few times).

That being said lets try to drop rare features and also try to deduplicate similar features using first-k-chars. as a hash.

First we'll read in data, lowercase everything and remove any whitespace

In [30]:
features = train_test[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])

In [31]:
features.head()

In [32]:
# count features and drop features with less than n counts

n = 5

feature_counts = Counter()
for feature in features.features:
    feature_counts.update(feature)
feature = sorted([k for (k,v) in feature_counts.items() if v > n])
feature[:10]


Notice that we see like variations of `24-hour` and we will see later that thats not the only duplicate features 

## Hashing cleaned up data using first4 characters.

We will first do some manual work to simplify the strings and use the first four characters as a key for each feature. then we will use that key to deduce data.

In [33]:
def clean(s):
    x = s.replace("-", "")
    x = x.replace(" ", "")
    x = x.replace("twenty four hour", "24")
    x = x.replace("24/7", "24")
    x = x.replace("24hr", "24")
    x = x.replace("24-hour", "24")
    x = x.replace("24hour", "24")
    x = x.replace("24 hour", "24")
    x = x.replace("common", "cm")
    x = x.replace("concierge", "doorman")
    x = x.replace("bicycle", "bike")
    x = x.replace("private", "pv")
    x = x.replace("deco", "dc")
    x = x.replace("decorative", "dc")
    x = x.replace("onsite", "os")
    x = x.replace("outdoor", "od")
    x = x.replace("ss appliances", "stainless")
    return x

def feature_hash(x):
    cleaned = clean(x, uniq)
    key = cleaned[:4].strip()
    return key

In [34]:
key2original = defaultdict(list)
k = 4
for f in feature:
    cleaned = clean(f)
    key = cleaned[:k].strip()
    key2original[key].append(f)

Lets take a look at the dedups! Don't worry about the key, but just take a look at what values are in the same key

In [35]:
key2original

In [36]:
print("number of deduped features:", len(key2original))
print("number of old features:", len(feature))

In order to make this easier to use, I'll output this as a CSV of the original feature and the deduced string.

In [45]:
def to_tuples():
    for f in feature:
        key = clean(f)[:k].strip()
        yield (f, key2original[key][0])
        
deduped = list(to_tuples())
df = pd.DataFrame(deduped, columns=["original_feature", "unique_feature"])

In [46]:
df.head()

In [47]:
df.to_csv("feature_deduplication.csv", index=False)